### Import libraries

In [ ]:
import numpy as np
import pandas as pd
import pywt
import cv2    
import os
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
from sklearn.metrics import roc_curve, auc, confusion_matrix, roc_auc_score, classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.applications import VGG16

# Remove some unwanted warnings
import logging
logging.getLogger('tensorflow').disabled = True 

### Plotting function

In [ ]:
from IPython.display import clear_output

class PlotLearning(keras.callbacks.Callback):
    """
    Callback to plot the learning curves of the model during training.
    """
    def on_train_begin(self, logs={}):
        self.metrics = {}
        for metric in logs:
            self.metrics[metric] = []
            

    def on_epoch_end(self, epoch, logs={}):
        # Storing metrics
        for metric in logs:
            if metric in self.metrics:
                self.metrics[metric].append(logs.get(metric))
            else:
                self.metrics[metric] = [logs.get(metric)]
        
        # Plotting
        metrics = [x for x in logs if 'val' not in x]
        
        f, axs = plt.subplots(1, len(metrics), figsize=(15,5))
        clear_output(wait=True)

        for i, metric in enumerate(metrics):
            axs[i].plot(range(1, epoch + 2), 
                        self.metrics[metric], 
                        label=metric)
            if logs['val_' + metric]:
                axs[i].plot(range(1, epoch + 2), 
                            self.metrics['val_' + metric], 
                            label='val_' + metric)
                
            axs[i].legend()
            axs[i].grid()

        plt.tight_layout()
        plt.show()

In [ ]:
callbacks_list = [PlotLearning()]

### Download data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# download content from Kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json
! kaggle datasets download maciejgronczynski/biggest-genderface-recognition-dataset
! kaggle datasets download rashikrahmanpritom/gender-recognition-dataset

# unzip packs
! unzip biggest-genderface-recognition-dataset.zip
! unzip gender-recognition-dataset.zip

# manage paths
!mv faces test_data
!mv Train/Train train_data
!mv Test/Test val_data

#### Train set

In [ ]:
subdir = ['Male','Female']
path = '/content/train_data/'

# Creating train set
x_train, y_train = [], []
for gender in subdir:
    for img_name in os.listdir(path+gender): 
        img = cv2.imread(path+gender+'/'+img_name) 
        x_train.append(img)
        if gender=='Male':
            y_train.append(1)
        else:
            y_train.append(0)

#### Validation set

In [ ]:
subdir = ['Male','Female']
path = '/content/val_data/'

x_val, y_val = [], []
for gender in subdir:
    for img_name in os.listdir(path+gender):
        img = cv2.imread(path+gender+'/'+img_name)

        x_val.append(img)
        if gender=='Male':
            y_val.append(1)
        else:
            y_val.append(0)

#### Test  set

In [ ]:
subdir = ['man','woman']
path = '/content/test_data/'

x_test, y_test = [], []
for gender in subdir:
    for img_name in os.listdir(path+gender):
        img = cv2.imread(path+gender+'/'+img_name)
        img = cv2.resize(img, (100, 100)) 
        x_test.append(img)
        
        if gender=='man':
            y_test.append(1)
        else:
            y_test.append(0)

### Preprocessing Rescaling

In [ ]:
x_train = np.array(x_train).astype('float32')/255
x_val = np.array(x_val).astype('float32')/255
x_test = np.array(x_test).astype('float32')/255

y_train = np.array(y_train).astype(int)
y_val = np.array(y_val).astype(int)
y_test = np.array(y_test).astype(int)

### Baseline Model

In [ ]:
model = Sequential()

model.add(layers.Input((100, 100, 3)))
model.add(layers.Conv2D(16, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2)))
model.add(layers.Conv2D(16, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

#### Training

In [ ]:
base_hist = model.fit(x_train, y_train,
                      validation_data=(x_val, y_val),
                      epochs=25, 
                      batch_size=32, 
                      verbose=1,
                      callbacks=callbacks_list)

### Fine-tuned Model

#### Data augmentation

In [ ]:
generator_3 = tf.keras.preprocessing.image.ImageDataGenerator(width_shift_range=0.1, 
                                 height_shift_range=0.1, 
                                 rotation_range=10, 
                                 zoom_range=0.1,
                                 horizontal_flip=True)
x_train_1 = generator_3.flow(x_train, y_train)

#### Neural network structure

In [ ]:
model_3 = Sequential()

model_3.add(layers.Input((100, 100, 3)))

model_3.add(layers.Conv2D(16, (3, 3), activation='relu'))
model_3.add(layers.Conv2D(16, (3, 3), activation='relu'))
model_3.add(layers.MaxPooling2D((2)))

model_3.add(layers.Conv2D(32, (3, 3), activation='relu'))
model_3.add(layers.Conv2D(32, (3, 3), activation='relu'))
model_3.add(layers.MaxPooling2D((2)))

model_3.add(layers.Flatten())
model_3.add(layers.Dense(64, activation='relu'))
model_3.add(layers.Dropout(0.5))
model_3.add(layers.Dense(64, activation='relu'))
model_3.add(layers.Dropout(0.5))
model_3.add(layers.Dense(1, activation='sigmoid'))

model_3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


#### Training

In [ ]:
history_6 = model_3.fit(x_train_1,
                        epochs=50,
                        batch_size=64,
                        verbose=1,
                        validation_data=(x_val, y_val),
                        callbacks=callbacks_list)

### Transfer Learning

In [ ]:
vggmodel = VGG16(weights='imagenet', include_top=False, input_shape=(100, 100, 3))
vggmodel.summary()

In [ ]:
# freeze every layer
vggmodel.trainable = False

# chain up the vgg16 architecture and the fully connected layer of the baseline model
last = vggmodel.layers[-1].output
x = layers.Flatten()(last)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation='sigmoid')(x)
model_vgg = Model(vggmodel.input, output)
model_vgg.summary()

model_vgg.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
Batch_size = 32
hist_vgg = model_vgg.fit(x_train_1,
                         batch_size=32,
                         steps_per_epoch=len(x_train)//Batch_size, 
                         epochs=25, 
                         validation_data=(x_val, y_val), 
                         validation_steps=len(x_val)//Batch_size, 
                         callbacks=callbacks_list)

In [ ]:
histV = hist_vgg.history
round(sum(histV['val_accuracy'])/len(histV['val_accuracy']), 3)

### Predictions

#### On validation set

In [ ]:
# baseline model
y_val_base_pred = model.predict(x_val)
y_val_base_pred_label = (y_val_base_pred > 0.5).astype(np.int)
# y_val_base_pred_label

# tuned model
y_val_tune_pred = model_3.predict(x_val)
y_val_tune_pred_label = (y_val_base_pred > 0.5).astype(np.int)
# y_val_tune_pred_label

# tf model
y_val_tf_pred = model_vgg.predict(x_val)
y_val_tf_pred_label = (y_val_tf_pred > 0.5).astype(np.int)
# y_val_tf_pred_label

#### On test set

In [ ]:
# baseline model
y_test_base_pred = model.predict(x_test)
y_test_base_pred_label = (y_test_base_pred > 0.5).astype(np.int)
# y_test_base_pred_label

# tuned model
y_test_tune_pred = model_3.predict(x_test)
y_test_tune_pred_label = (y_test_tune_pred > 0.5).astype(np.int)
# y_test_tune_pred_label

# tf model
y_test_tf_pred = model_vgg.predict(x_test)
y_test_tf_pred_label = (y_test_tf_pred > 0.5).astype(np.int)
# y_test_tf_pred_label

### Evaluation

#### Loss & Accuracy (Train + Validation Set)

In [ ]:
header = ['loss', 'accuracy', 'val_loss', 'val_accuracy']
index = ['Base Model', 'Fine Tuned Model', 'Transfer Learning Model']
histB = base_hist.history
histT = history_6.history
histV = hist_vgg.history

hist = np.array([histB, histT, histV])
data = []
for i in range(len(index)):
    temp = []
    # get the average acc & loss over all epochs
    for h in range(len(header)):
        temp.append(round(sum(hist[i][header[h]]) / len(hist[i][header[h]]), 7))
    data.append(temp)

loss_acc = pd.DataFrame(data, columns=header, index=index)
print(loss_acc)

#### ROC & AUC 

In [ ]:
val_preds = [y_val_base_pred_label, y_val_tune_pred_label, y_val_tf_pred_label]
test_preds = [y_test_base_pred_label, y_test_tune_pred_label, y_test_tf_pred_label]

# plot ROC
def plot_roc(fpr, tpr, i, dataset, auc_score):
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % auc_score)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.title('Model {} ROC Curve ({} set)'.format(i, dataset))
    plt.xlabel('False Positive Rate (Positive label: 1)') 
    plt.ylabel('True Positive Rate (Positive label: 1)') 
    plt.legend(loc='lower right')
    plt.show()
    
def roc_auc(y_true, y_pred, dataset):
    for i in range(len(y_pred)):
        fpr, tpr, thresholds = roc_curve(y_true, y_pred[i])
        auc_score = roc_auc_score(y_true, y_pred[i])
        plot_roc(fpr, tpr, i, dataset, auc_score)
        #print('Model {} AUC score on {} set: {}'.format(i, dataset, round(auc_score, 3)))

In [ ]:
roc_auc(y_val, val_preds, 'validation')

In [ ]:
roc_auc(y_test, test_preds, 'test')

#### Confusion matrix + Classification report

In [ ]:
def confusion_classification(y_true, y_pred):
    for i in range(len(y_pred)):
        print('Model {} confusion matrix'.format(i))
        print(tf.math.confusion_matrix(y_true, y_pred[i]))
        print('')
        print('Model {} classification_report'.format(i))
        print(classification_report(y_true, y_pred[i]))
        print('')
        print('')

In [ ]:
# validation set
confusion_classification(y_val, val_preds)

In [ ]:
# test set
confusion_classification(y_test, test_preds)

In [ ]:
# baseline validation confusion matrix
plt.figure(figsize=(5, 4), dpi=100)
ax = plt.axes()
cm_base_val = confusion_matrix(y_val, y_val_base_pred_label.ravel())
sns.heatmap(cm_base_val, annot = True, fmt = '.20g')
ax.set_title('Baseline Model Confusion Matrix on Validation Data')
plt.xticks()
plt.show()

In [ ]:
# baseline test confusion matrix
plt.figure(figsize=(5, 4), dpi=100)
ax = plt.axes()
cm_base_test = confusion_matrix(y_test, y_test_base_pred_label.ravel())
sns.heatmap(cm_base_test, annot = True, fmt = '.20g')
ax.set_title('Baseline Model Confusion Matrix on Test Data')
plt.show()

In [ ]:
# fine-tuning validation confusion matrix
plt.figure(figsize=(5, 4), dpi=100)
ax = plt.axes()
cm_best_val = confusion_matrix(y_val, y_val_best_pred_label.ravel())
sns.heatmap(cm_best_val, annot = True, fmt = '.20g')
ax.set_title(' Fine-tuned Model Confusion Matrix on Validation Data')
plt.xticks()
plt.show()

In [ ]:
# fine-tuning test confusion matrix
plt.figure(figsize=(5, 4), dpi=100)
ax = plt.axes()
cm_best_test = confusion_matrix(y_test, y_test_best_pred_label.ravel())
sns.heatmap(cm_best_test, annot = True, fmt = '.20g')
ax.set_title('Fine-tuned Model Confusion Matrix on Test Data')
plt.show()

In [ ]:
# transfer learning validation confusion matrix
plt.figure(figsize=(5, 4), dpi=100)
ax = plt.axes()
cm_vgg_val = confusion_matrix(y_val, y_val_vgg_pred_label.ravel())
sns.heatmap(cm_vgg_val, annot = True, fmt = '.20g')
ax.set_title('Vgg Model Confusion Matrix on Validation Data')
plt.xticks()
plt.show()

In [ ]:
# transfer learning test confusion matrix
plt.figure(figsize=(5, 4), dpi=100)
ax = plt.axes()
cm_vgg_test = confusion_matrix(y_test, y_test_vgg_pred_label.ravel())
sns.heatmap(cm_base_test, annot = True, fmt = '.20g')
ax.set_title('Vgg Model Confusion Matrix on Test Data')
plt.xticks()
plt.show()